<a href="https://colab.research.google.com/github/kiranbeethoju/CHAI/blob/master/stock_predictions_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [171]:
import datetime
import pandas as pd
import numpy as np
import pylab as pl
import datetime
from sklearn.cluster import AffinityPropagation
from sklearn import metrics
from matplotlib.collections import LineCollection
from pandas_datareader import data as wb
from sklearn import cluster, covariance, manifold

start = '1995-02-01'
end = '2020-06-25'

tickers = ['HINDPETRO.NS']

thelen = len(tickers)

price_data = []
for ticker in tickers:
    prices = wb.DataReader(ticker, start = start, end = end, data_source='yahoo')[['Open','Adj Close']]
    price_data.append(prices.assign(ticker=ticker)[['ticker', 'Open', 'Adj Close']])

#names = np.reshape(price_data, (len(price_data), 1))

names = pd.concat(price_data)
names.reset_index()

#pd.set_option('display.max_columns', 500)

open = np.array([q['Open'] for q in price_data]).astype(np.float)
close = np.array([q['Adj Close'] for q in price_data]).astype(np.float)
print(names)


                  ticker        Open   Adj Close
Date                                            
1996-01-02  HINDPETRO.NS   43.703701    1.294188
1996-01-03  HINDPETRO.NS   42.963001    1.285323
1996-01-05  HINDPETRO.NS   42.222198    1.294188
1996-01-08  HINDPETRO.NS   43.259300    1.245654
1996-01-09  HINDPETRO.NS   41.481499    1.262718
...                  ...         ...         ...
2020-06-22  HINDPETRO.NS  225.949997  228.137451
2020-06-23  HINDPETRO.NS  239.000000  227.181305
2020-06-24  HINDPETRO.NS  238.000000  218.384705
2020-06-25  HINDPETRO.NS  228.449997  212.121933
2020-06-26  HINDPETRO.NS  226.949997  221.109741

[6040 rows x 3 columns]


In [172]:
names = names.reset_index()
open = names.rename(columns={'Date':'ds','Open':'y'})
open = open[['ds','y']]
close = names.rename(columns={'Date':'ds','Adj Close':'y'})
close = close[["ds","y"]]

In [173]:
close

,ds,y
0,1996-01-02,1.294188
1,1996-01-03,1.285323
2,1996-01-05,1.294188
3,1996-01-08,1.245654
4,1996-01-09,1.262718
...,...,...
6035,2020-06-22,228.137451
6036,2020-06-23,227.181305
6037,2020-06-24,218.384705
6038,2020-06-25,212.121933


In [174]:
!pip install fbprophet

In [175]:
from fbprophet import Prophet
df = open
df = df[5900:]
df.tail()

,ds,y
6035,2020-06-22,225.949997
6036,2020-06-23,239.000000
6037,2020-06-24,238.000000
6038,2020-06-25,228.449997
6039,2020-06-26,226.949997


In [176]:
#df = df[df['ds'].dt.dayofweek < 5]
m = Prophet()
m.fit(df)


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [198]:
future = m.make_future_dataframe(periods=400)
future.tail()
forecast = m.predict(future)
future = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

In [199]:
future = future.rename(columns={"yhat":"yhat_future","yhat_lower":"yhat_lower_future","yhat_upper":"yhat_upper_future"})

In [206]:
mixed = pd.merge(df,future,on="ds",how="inner")

In [207]:
mixed.tail(30)

,ds,y,yhat_future,yhat_lower_future,yhat_upper_future
110,2020-05-15,191.000000,196.469852,181.880243,209.730274
111,2020-05-18,189.500000,199.990192,186.669548,214.373574
112,2020-05-19,176.550003,200.112936,184.748903,213.272229
113,2020-05-20,172.000000,199.923841,186.209020,215.235150
114,2020-05-21,180.000000,200.132712,185.367304,213.511325
115,2020-05-22,180.600006,198.396399,183.689247,212.334311
116,2020-05-26,183.000000,202.785754,188.490180,217.844654
117,2020-05-27,181.000000,202.596659,189.131336,218.144035
118,2020-05-28,186.949997,202.805530,187.597628,217.001494
119,2020-05-29,184.649994,201.069217,186.760985,215.209122


In [208]:
avg = (mixed["yhat_future"].iloc[0]+mixed["yhat_lower_future"].iloc[0]+mixed["yhat_upper_future"].iloc[0])/3

In [209]:
mixed["avg_val"] = ''
for i in range(0, len(mixed)):
  avg = (mixed["yhat_future"].iloc[i]+mixed["yhat_lower_future"].iloc[i]+mixed["yhat_upper_future"].iloc[i])/3
  mixed['avg_val'].iloc[i] = avg

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [210]:
mixed = mixed
import plotly as py
from plotly.graph_objs import *


In [214]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=mixed["ds"], y=mixed["y"],
                    mode='lines',
                    name='Actual Opening Values'))
fig.add_trace(go.Scatter(x=mixed["ds"], y=mixed["yhat_future"],
                    mode='lines+markers',
                    name='Predicted Opening Values'))
fig.show()
fig.add_trace(go.Scatter(x=mixed["ds"], y=mixed["y"],
                    mode='lines',
                    name='Actual Opening Values'))
fig.add_trace(go.Scatter(x=mixed["ds"], y=mixed["avg_val"],
                    mode='markers',
                    name='Predicted Opening Avg Values '))

fig.show()